In [1]:
import itertools
import sys
import math

from sklearn.model_selection import train_test_split, KFold


data_path = '/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/data/'

In [352]:
%run /Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/lib.py

In [353]:
train = Data()
train.read(data_path, 'train')

In [354]:
def expand_grid(dictionary):
    return pd.DataFrame([row for row in itertools.product(*dictionary.values())], 
                       columns=dictionary.keys())

In [355]:
def rmse(x, y):
    n = len(x)
    return math.sqrt(1.0 * sum((x - y)**2) / n)

In [356]:
def cross_validation(model, data, n_folds=5, n_iters=5, seed=2707, verbose=1):
    scores = []
    user_ids = data.edu['uid'].unique()
    
    for iter in range(n_iters):
        kf = KFold(n_splits=n_folds, random_state=seed+iter, shuffle=True)
        print(kf)
        fold = 0
        for train_index, test_index in kf.split(user_ids):
            fold += 1
            if verbose >= 2:
                print('Prepare data: %s' % datetime.datetime.now())
                
            data_train = data.get(user_ids[train_index])
            data_test = data.get(user_ids[test_index])
            test_age = data_test.edu['age'].values
            data_test.edu.drop(['age'], 1, inplace=True)

            if verbose >= 2:
                print('Fit: %s' % datetime.datetime.now())
            model.prepare(data_train, data_test)
            model.fit(data_train)
            if verbose >= 2:
                print('Predict: %s' % datetime.datetime.now())

            assert 'age' not in data_test.edu.columns
            pred = model.predict(data_test)
            if verbose >= 2:
                print('Auc: %s' % datetime.datetime.now())
            score = rmse(test_age, pred['res'].values)
            scores.append(score)

            #if verbose >= 1:
            print('%2d -%2d : %1.4f, mean=%1.4f' % (iter, fold, score, np.mean(scores)))
    return np.mean(scores)

In [357]:
grid = expand_grid({
    'boosting_type': ['gbdt'],
    'min_data_in_leaf': [25],
    'lambda_l2': [0.06],
    'num_leaves': [17],
    'learning_rate': [0.035],
    'feature_fraction': [1],
    'bagging_fraction': [1],
    'bagging_freq': [5],
    'num_boost_round': [400],
})
min_rmse = 1e10
best_params = None
for i, r in grid.iterrows():
    model = LgbModel({
        'boosting_type': r.boosting_type,
        'min_data_in_leaf': r.min_data_in_leaf,
        'lambda_l2': r.lambda_l2,
        'num_leaves': r.num_leaves,
        'learning_rate': r.learning_rate,
        'feature_fraction': r.feature_fraction,
        'bagging_fraction': r.bagging_fraction,
        'bagging_freq': r.bagging_freq,
        'num_boost_round': r.num_boost_round,
    })
    rr = cross_validation(model, train, n_folds=5, n_iters=1, verbose=0)
    if rr < min_rmse:
        min_rmse = rr
        best_params = r.copy()
    print(r)
    print(rr)
        

KFold(n_splits=5, random_state=2707, shuffle=True)


/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1111: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


Starting train: 2021-07-27 00:10:10.910770
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000486 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1472
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 17
[LightGBM] [Info] Start training from score 35.017179


/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1113: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


 0 - 1 : 8.5001, mean=8.5001
Starting train: 2021-07-27 00:10:28.805122
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000485 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1478
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 17
[LightGBM] [Info] Start training from score 34.965302
 0 - 2 : 8.5237, mean=8.5119
Starting train: 2021-07-27 00:10:54.153422
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000583 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1474
[LightGBM] [Info] Number of data points in the train set: 23575, number of used features: 17
[LightGBM] [Info] Start training from score 35.056161
 0 - 3 : 8.3497, mean=8.4578
Startin

In [351]:
print(min_rmse)
print(best_params)

8.482336010597404
boosting_type        gbdt
min_data_in_leaf       25
lambda_l2            0.06
num_leaves             17
learning_rate       0.035
feature_fraction        1
bagging_fraction        1
bagging_freq            5
num_boost_round       400
Name: 0, dtype: object


In [215]:
prod_model = LgbModel({
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 25,
    'lambda_l2': 0.06,
    'num_leaves': 17,
    'learning_rate': 0.035,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'num_boost_round': 400,
})
prod_model.prepare(train, None)
prod_model.fit(train)
prod_model.model.save_model('/Users/tyamgin/Projects/mlbootcamp/vkcup2021_2/models/lgb.txt')
prod_model.predict(train)

/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Starting train: 2021-07-26 22:03:06.492044
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002807 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1186
[LightGBM] [Info] Number of data points in the train set: 29469, number of used features: 15
[LightGBM] [Info] Start training from score 35.029149


,uid,res
0,356,45.293078
1,26356,40.299689
2,97,36.529859
3,319,37.234419
4,30288,32.221299
...,...,...
29464,33331,54.447875
29465,119788,40.710635
29466,97581,36.045508
29467,104038,69.633122


In [58]:
a = train.edu.copy()

In [87]:
[
 np.median([
     train.registered_year_by_uid[fr]
     for fr in train.friends.get(uid, [])
     if fr in train.registered_year_by_uid
 ])
 for uid in a['uid'].values
]

[2009.0,
 2012.0,
 2010.0,
 2011.0,
 2010.0,
 2013.0,
 2013.0,
 2010.0,
 2011.0,
 2010.0,
 2014.0,
 2012.0,
 2011.0,
 2010.0,
 2012.0,
 2019.0,
 2011.0,
 2015.5,
 2010.0,
 2013.0,
 2010.0,
 2011.0,
 2015.0,
 2014.0,
 2011.0,
 nan,
 nan,
 2010.0,
 nan,
 nan,
 2011.0,
 2012.5,
 2011.0,
 2014.0,
 2011.0,
 2011.0,
 2009.0,
 2015.0,
 2012.0,
 2018.0,
 2013.0,
 2014.5,
 2011.0,
 nan,
 2014.0,
 nan,
 2011.0,
 2018.0,
 2009.0,
 nan,
 2014.0,
 2010.0,
 2012.5,
 2008.5,
 2012.0,
 2010.0,
 2009.5,
 nan,
 2013.0,
 2018.0,
 2012.0,
 2014.0,
 2010.0,
 2010.0,
 2016.0,
 2011.0,
 2010.0,
 2009.5,
 2011.0,
 2013.0,
 2014.0,
 2016.5,
 2014.0,
 2010.0,
 2016.0,
 2013.0,
 2015.5,
 2011.0,
 2011.0,
 2011.0,
 2013.5,
 2010.0,
 2018.0,
 2011.0,
 2008.0,
 2014.0,
 nan,
 2010.5,
 2009.5,
 2015.0,
 2015.0,
 2011.0,
 2012.0,
 2013.0,
 2014.0,
 2015.5,
 2016.0,
 2010.0,
 2018.0,
 2011.0,
 nan,
 2017.0,
 2011.0,
 2012.0,
 nan,
 2014.5,
 nan,
 2009.0,
 2014.0,
 2013.0,
 2013.0,
 2012.0,
 2012.0,
 2010.0,
 nan,
 201

In [79]:
np.median

<function numpy.median(a, axis=None, out=None, overwrite_input=False, keepdims=False)>

In [83]:
np.median([])

nan

In [97]:
dir(train)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_prepare_friends',
 '_prepare_groups',
 'edu',
 'friends',
 'get',
 'groups',
 'read',
 'registered_year_by_uid']

In [104]:
'ag1e' in train.edu.columns

False

In [112]:
a = pd.read_csv(os.path.join(data_path, f'train.csv')).drop(['age'], 1)
a.to_csv(os.path.join(data_path, f'test.csv'), header=True, index=False)

In [122]:
train.edu

,uid,school_education,graduation_1,graduation_2,graduation_3,graduation_4,graduation_5,graduation_6,graduation_7,age,registered_year
0,356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48,2017
1,26356,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,NaN,49,2008
2,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,2011
3,319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,2020
4,30288,2008.0,NaN,NaN,NaN,NaN,2013.0,NaN,NaN,30,2008
...,...,...,...,...,...,...,...,...,...,...,...
29464,33331,1987.0,NaN,NaN,NaN,NaN,1991.0,NaN,NaN,51,2008
29465,119788,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,2013
29466,97581,2002.0,NaN,NaN,NaN,NaN,2007.0,NaN,NaN,36,2010
29467,104038,1969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,2013


In [181]:
list(train.groups.items())[0]

(34673,
 array([  7189,  12606,  25510,  43401,  52467,  59728,  74127,  74134,
        110624, 135972, 148721, 157789, 157912, 161477, 168378, 170127,
        170187, 190191, 213872, 219293, 228244, 240885, 251760, 257141,
        262585, 266111, 276925, 287893, 298636, 298638, 300538, 306743,
        335045, 336892, 338667, 342334, 342343, 344140]))

In [292]:
np.median([])

nan

In [295]:
np.median([np.nan, np.nan, 1])

nan